# Notebook 4 — Pan-Cancer AlphaMissense HRR Analysis (v2 FIX)

**Critical fix (2026-02-16):** The cBioPortal API changed behavior — `entrezGeneIds: []` now returns zero mutations instead of all mutations. Fixed by passing explicit Entrez Gene IDs for HRR genes, with fallback strategies.

**Changes from FIXED v1:**
1. `fetch_mutations()` now uses `HRR_ENTREZ_IDS` (explicit gene list) instead of `[]`
2. Added fallback: alternate sample list IDs + gene-by-gene query
3. Added diagnostic print for API response status


In [ ]:
# ============================================================
# 1. SETUP — GLOBAL SEEDS + PACKAGES
# ============================================================
import subprocess, sys
for p in ["pandas","numpy","matplotlib","seaborn","scipy","lifelines","requests","tqdm","scikit-learn"]:
    subprocess.check_call([sys.executable,"-m","pip","install","-q",p])

# ── GLOBAL SEEDS (MUST be first) ──
import random
import numpy as np
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

import pandas as pd
import matplotlib; matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
from tqdm.auto import tqdm
import requests, io, json, re, warnings, time
warnings.filterwarnings('ignore')
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test, multivariate_logrank_test
from sklearn.metrics import cohen_kappa_score, confusion_matrix

DATA_DIR = Path("data"); RESULTS_DIR = Path("results"); FIG_DIR = Path("figures")
for d in [DATA_DIR, RESULTS_DIR, FIG_DIR, DATA_DIR/"pancancer"]:
    d.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({'font.family':'sans-serif','font.size':10,'figure.dpi':300,
                     'savefig.dpi':300,'savefig.bbox':'tight'})

# Package versions for reproducibility
print("=== PACKAGE VERSIONS ===")
for pkg_name in ["pandas","numpy","scipy","lifelines","sklearn","matplotlib","seaborn"]:
    try:
        mod = __import__(pkg_name)
        print(f"  {pkg_name}: {mod.__version__}")
    except:
        pass
print(f"  Python: {sys.version}")
print(f"  Random seed: {SEED}")
print(f"  Data download date: {pd.Timestamp.now().strftime('%Y-%m-%d')}")

# Gene lists
COHORT_A = ["BRCA1","BRCA2","ATM"]
COHORT_B = ["PALB2","BRIP1","BARD1","CDK12","CHEK1","CHEK2","FANCL",
            "RAD51B","RAD51C","RAD51D","RAD54L"]
EXPANDED = ["FANCA","FANCC","FANCD2","FANCE","FANCF","FANCG",
            "NBN","MRE11","RAD50","ATR","ATRX"]
HRR_ALL = sorted(set(COHORT_A + COHORT_B + EXPANDED))

GENE_TO_UNIPROT = {
    "BRCA1":"P38398","BRCA2":"P51587","ATM":"Q13315","PALB2":"Q86YC2",
    "BRIP1":"Q9BX63","BARD1":"Q99728","CDK12":"Q9NYV4","CHEK1":"O14757",
    "CHEK2":"O96017","FANCL":"Q9NW38","RAD51B":"O15315","RAD51C":"O43502",
    "RAD51D":"O75771","RAD54L":"Q92698","FANCA":"O15360","FANCC":"Q00597",
    "FANCD2":"Q9BXW9","FANCE":"Q9HB96","FANCF":"Q9NPI8","FANCG":"O15287",
    "NBN":"O60934","MRE11":"P49959","RAD50":"Q92878","ATR":"Q13535","ATRX":"P46100",
}

TCGA_STUDIES = [
    ("brca_tcga_pan_can_atlas_2018", "Breast"),
    ("ov_tcga_pan_can_atlas_2018", "Ovarian"),
    ("paad_tcga_pan_can_atlas_2018", "Pancreas"),
    ("prad_tcga_pan_can_atlas_2018", "Prostate"),
    ("blca_tcga_pan_can_atlas_2018", "Bladder"),
    ("ucec_tcga_pan_can_atlas_2018", "Endometrial"),
    ("luad_tcga_pan_can_atlas_2018", "Lung Adeno"),
    ("lusc_tcga_pan_can_atlas_2018", "Lung Squamous"),
    ("coadread_tcga_pan_can_atlas_2018", "Colorectal"),
    ("stad_tcga_pan_can_atlas_2018", "Gastric"),
    ("hnsc_tcga_pan_can_atlas_2018", "Head & Neck"),
    ("skcm_tcga_pan_can_atlas_2018", "Melanoma"),
    ("lihc_tcga_pan_can_atlas_2018", "Liver"),
    ("chol_tcga_pan_can_atlas_2018", "Cholangiocarcinoma"),
    ("esca_tcga_pan_can_atlas_2018", "Esophageal"),
    ("kirc_tcga_pan_can_atlas_2018", "Kidney ccRCC"),
    ("kirp_tcga_pan_can_atlas_2018", "Kidney Papillary"),
    ("gbm_tcga_pan_can_atlas_2018", "Glioblastoma"),
    ("lgg_tcga_pan_can_atlas_2018", "Low Grade Glioma"),
    ("sarc_tcga_pan_can_atlas_2018", "Sarcoma"),
    ("thca_tcga_pan_can_atlas_2018", "Thyroid"),
    ("cesc_tcga_pan_can_atlas_2018", "Cervical"),
    ("acc_tcga_pan_can_atlas_2018", "Adrenocortical"),
    ("meso_tcga_pan_can_atlas_2018", "Mesothelioma"),
    ("uvm_tcga_pan_can_atlas_2018", "Uveal Melanoma"),
    ("dlbc_tcga_pan_can_atlas_2018", "DLBCL"),
    ("tgct_tcga_pan_can_atlas_2018", "Testicular"),
    ("thym_tcga_pan_can_atlas_2018", "Thymoma"),
    ("pcpg_tcga_pan_can_atlas_2018", "Pheochromocytoma"),
    ("kich_tcga_pan_can_atlas_2018", "Kidney Chromophobe"),
    ("ucs_tcga_pan_can_atlas_2018", "Uterine Carcinosarcoma"),
]

print(f"\nSetup complete. {len(TCGA_STUDIES)} TCGA studies. Seed={SEED}.")


In [ ]:
# ============================================================
# 2. LOAD ALPHAMISSENSE LOOKUP (from Notebook 1)
# ============================================================
am_path = DATA_DIR / "processed" / "alphamissense_hrr_genes.csv"
if am_path.exists():
    df_am = pd.read_csv(am_path)
    am_lookup = {}
    for _, row in df_am.iterrows():
        key = f"{row['uniprot_id']}_{row['protein_variant']}"
        am_lookup[key] = (row['am_pathogenicity'], row['am_class'])
    print(f"AlphaMissense lookup: {len(am_lookup):,} variants")
else:
    print("ERROR: Run Notebook 1 first to generate AlphaMissense lookup!")
    am_lookup = {}


In [ ]:
# ============================================================
# 3. DOWNLOAD + FILTER + ANNOTATE — ALL TCGA STUDIES
# ============================================================

CBIO_API = "https://www.cbioportal.org/api"

aa3to1 = {'Ala':'A','Arg':'R','Asn':'N','Asp':'D','Cys':'C','Gln':'Q',
          'Glu':'E','Gly':'G','His':'H','Ile':'I','Leu':'L','Lys':'K',
          'Met':'M','Phe':'F','Pro':'P','Ser':'S','Thr':'T','Trp':'W',
          'Tyr':'Y','Val':'V','Ter':'*','Sec':'U'}

def parse_protein(val):
    if pd.isna(val): return None
    s = str(val).strip()
    m3 = re.match(r'p\.([A-Z][a-z]{2})(\d+)([A-Z][a-z]{2})', s)
    if m3:
        r, alt = aa3to1.get(m3.group(1)), aa3to1.get(m3.group(3))
        if r and alt and r != alt: return (r, int(m3.group(2)), alt)
    m1 = re.match(r'p\.([A-Z*])(\d+)([A-Z*])', s)
    if m1 and m1.group(1) != m1.group(3):
        return (m1.group(1), int(m1.group(2)), m1.group(3))
    mb = re.match(r'^([A-Z])(\d+)([A-Z])$', s)
    if mb and mb.group(1) != mb.group(3):
        return (mb.group(1), int(mb.group(2)), mb.group(3))
    return None

# Entrez Gene IDs for HRR genes (NCBI Gene)
HRR_ENTREZ = {
    "BRCA1":672,"BRCA2":675,"ATM":472,"PALB2":79728,"BRIP1":83990,
    "BARD1":580,"CDK12":51755,"CHEK1":1111,"CHEK2":11200,"FANCL":55120,
    "RAD51B":5890,"RAD51C":5889,"RAD51D":5892,"RAD54L":8438,
    "FANCA":2175,"FANCC":2176,"FANCD2":2177,"FANCE":2178,"FANCF":2188,
    "FANCG":2189,"NBN":4683,"MRE11":4361,"RAD50":10111,"ATR":545,
    "ATRX":546,
}
HRR_ENTREZ_IDS = list(HRR_ENTREZ.values())

def fetch_mutations(study_id):
    """Fetch mutations from cBioPortal API.
    
    FIX (2026-02-16): The cBioPortal API now interprets entrezGeneIds=[] as
    "no genes" instead of "all genes". We use two strategies:
      1. Primary: fetch HRR genes by Entrez ID (faster, targeted)
      2. Fallback: fetch all mutations via /mutations endpoint
    """
    try:
        # Get molecular profile
        profiles = requests.get(
            f"{CBIO_API}/studies/{study_id}/molecular-profiles",
            headers={"Accept":"application/json"}, timeout=30
        ).json()
        mut_profile = next((p["molecularProfileId"] for p in profiles
                           if p["molecularAlterationType"]=="MUTATION_EXTENDED"), None)
        if not mut_profile:
            print(f"    No mutation profile found")
            return None

        # ── STRATEGY 1: Fetch HRR genes by Entrez ID ──
        resp = requests.post(
            f"{CBIO_API}/molecular-profiles/{mut_profile}/mutations/fetch",
            headers={"Accept":"application/json","Content-Type":"application/json"},
            json={"sampleListId": f"{study_id}_all",
                  "entrezGeneIds": HRR_ENTREZ_IDS},
            params={"projection":"DETAILED"}, timeout=180
        )
        if resp.status_code == 200:
            data = resp.json()
            if len(data) > 0:
                return pd.json_normalize(data)

        # ── STRATEGY 2: Fallback — try without sampleListId filter ──
        # Some studies use different sample list naming
        sample_lists = requests.get(
            f"{CBIO_API}/studies/{study_id}/sample-lists",
            headers={"Accept":"application/json"}, timeout=30
        ).json()
        alt_lists = [sl["sampleListId"] for sl in sample_lists
                     if "all" in sl.get("sampleListId","").lower()
                     or sl.get("category","") == "all_cases_in_study"]
        
        for sl_id in alt_lists:
            if sl_id == f"{study_id}_all":
                continue  # already tried
            resp2 = requests.post(
                f"{CBIO_API}/molecular-profiles/{mut_profile}/mutations/fetch",
                headers={"Accept":"application/json","Content-Type":"application/json"},
                json={"sampleListId": sl_id,
                      "entrezGeneIds": HRR_ENTREZ_IDS},
                params={"projection":"DETAILED"}, timeout=180
            )
            if resp2.status_code == 200:
                data2 = resp2.json()
                if len(data2) > 0:
                    print(f"    Used alternate sample list: {sl_id}")
                    return pd.json_normalize(data2)

        # ── STRATEGY 3: Last resort — gene-by-gene query ──
        print(f"    Trying gene-by-gene fetch...")
        all_muts = []
        for gene, eid in HRR_ENTREZ.items():
            try:
                r = requests.get(
                    f"{CBIO_API}/molecular-profiles/{mut_profile}/mutations",
                    headers={"Accept":"application/json"},
                    params={"sampleListId": f"{study_id}_all",
                            "entrezGeneId": eid,
                            "projection":"DETAILED"},
                    timeout=30
                )
                if r.status_code == 200:
                    muts = r.json()
                    if muts:
                        all_muts.extend(muts)
            except:
                pass
        if all_muts:
            return pd.json_normalize(all_muts)

        print(f"    All strategies returned 0 mutations")
        return None

    except Exception as e:
        print(f"    Mutation fetch failed: {e}")
    return None

def fetch_clinical(study_id):
    try:
        resp = requests.get(
            f"{CBIO_API}/studies/{study_id}/clinical-data",
            headers={"Accept":"application/json"},
            params={"clinicalDataType":"PATIENT","projection":"DETAILED"}, timeout=60
        ).json()
        df = pd.json_normalize(resp)
        if "clinicalAttributeId" in df.columns:
            return df.pivot_table(index="patientId", columns="clinicalAttributeId",
                                   values="value", aggfunc="first").reset_index()
        return df
    except:
        return pd.DataFrame()

def annotate_study(df_mut, study_id, tumor_name):
    gene_col = next((c for c in ["Hugo_Symbol","gene.hugoGeneSymbol","hugoGeneSymbol"]
                     if c in df_mut.columns), None)
    class_col = next((c for c in ["Variant_Classification","mutationType"]
                      if c in df_mut.columns), None)
    sample_col = next((c for c in ["Tumor_Sample_Barcode","sampleId"]
                       if c in df_mut.columns), None)
    hgvsp_col = next((c for c in ["HGVSp_Short","proteinChange","HGVSp"]
                      if c in df_mut.columns), None)
    patient_col = "patientId" if "patientId" in df_mut.columns else None
    if not all([gene_col, class_col, sample_col, hgvsp_col]):
        return pd.DataFrame()
    df_hrr = df_mut[df_mut[gene_col].isin(HRR_ALL)]
    df_miss = df_hrr[df_hrr[class_col].str.contains("issense", case=False, na=False)].copy()
    if len(df_miss) == 0: return pd.DataFrame()
    parsed = df_miss[hgvsp_col].apply(parse_protein)
    df_miss = df_miss[parsed.notna()].copy()
    parsed = parsed[parsed.notna()]
    if len(df_miss) == 0: return pd.DataFrame()
    df_miss["ref_aa"] = [p[0] for p in parsed]
    df_miss["protein_pos"] = [p[1] for p in parsed]
    df_miss["alt_aa"] = [p[2] for p in parsed]
    df_miss["gene"] = df_miss[gene_col]
    df_miss["sample_id"] = df_miss[sample_col]
    df_miss["patient_id"] = df_miss[patient_col] if patient_col else df_miss[sample_col]
    df_miss["uniprot_id"] = df_miss["gene"].map(GENE_TO_UNIPROT)
    scores, classes = [], []
    for _, row in df_miss.iterrows():
        key = f"{row.get('uniprot_id','')}_{row['ref_aa']}{row['protein_pos']}{row['alt_aa']}"
        if key in am_lookup:
            scores.append(am_lookup[key][0])
            classes.append(am_lookup[key][1])
        else:
            scores.append(np.nan)
            classes.append("not_found")
    df_miss["am_pathogenicity"] = scores
    df_miss["am_class"] = classes
    df_miss["study_id"] = study_id
    df_miss["tumor_type"] = tumor_name
    return df_miss[["study_id","tumor_type","sample_id","patient_id","gene",
                     "am_pathogenicity","am_class"]].copy()

# ── VALIDATED EVENT CODING ──
# FIX: Explicit mapping with validation, not heuristic string matching
def code_os_event(val):
    """Explicit OS_STATUS coding with validation.
    TCGA PanCan Atlas uses: '1:DECEASED' / '0:LIVING'
    Some studies use: 'DECEASED' / 'LIVING'
    """
    s = str(val).strip().upper()
    if s in ["1:DECEASED", "DECEASED", "DEAD"]:
        return 1
    elif s in ["0:LIVING", "LIVING", "ALIVE"]:
        return 0
    elif s.startswith("1:"):
        return 1
    elif s.startswith("0:"):
        return 0
    else:
        return np.nan  # UNKNOWN → missing, not guessed

# === MAIN LOOP ===
all_variants = []
all_clinical = {}
study_summary = []
os_status_values = {}  # Track unique OS_STATUS values per study for validation

for i, (sid, tumor) in enumerate(TCGA_STUDIES):
    print(f"\n[{i+1}/{len(TCGA_STUDIES)}] {tumor} ({sid})")
    df_mut = fetch_mutations(sid)
    if df_mut is None or len(df_mut) == 0:
        print(f"  No mutations — skipping")
        study_summary.append({"study_id":sid,"tumor":tumor,"n_mutations":0,
                              "n_hrr_missense":0,"n_patients":0})
        time.sleep(0.5)
        continue
    print(f"  Mutations: {len(df_mut):,}")
    df_ann = annotate_study(df_mut, sid, tumor)
    n_hrr = len(df_ann)
    n_pat = df_ann["patient_id"].nunique() if n_hrr > 0 else 0
    n_path = (df_ann["am_class"]=="pathogenic").sum() if n_hrr > 0 else 0
    print(f"  HRR missense: {n_hrr} in {n_pat} patients ({n_path} AM-pathogenic)")
    study_summary.append({"study_id":sid,"tumor":tumor,"n_mutations":len(df_mut),
                          "n_hrr_missense":n_hrr,"n_patients":n_pat})
    if n_hrr > 0:
        all_variants.append(df_ann)
    df_clin = fetch_clinical(sid)
    if len(df_clin) > 0:
        all_clinical[sid] = df_clin
        # Track OS_STATUS values for validation
        os_col = next((c for c in df_clin.columns if str(c).upper() == "OS_STATUS"), None)
        if os_col:
            unique_vals = df_clin[os_col].dropna().unique().tolist()
            os_status_values[tumor] = unique_vals
        print(f"  Clinical: {len(df_clin)} patients")
    time.sleep(1.0)

# Combine
if all_variants:
    df_all = pd.concat(all_variants, ignore_index=True)
    df_all.to_csv(RESULTS_DIR / "pancancer_hrr_variants.csv", index=False)
    print(f"\n{'='*60}")
    print(f"PAN-CANCER SUMMARY")
    print(f"{'='*60}")
    print(f"Total: {len(df_all)} HRR missense variants")
    print(f"Patients: {df_all['patient_id'].nunique()}")
    print(f"Tumor types: {df_all['tumor_type'].nunique()}")
    print(f"AM-Pathogenic: {(df_all['am_class']=='pathogenic').sum()}")
    print(f"AM-Benign: {(df_all['am_class']=='benign').sum()}")
else:
    df_all = pd.DataFrame()
    print("\nNo variants found")

# ── VALIDATE EVENT CODING ──
print("\n=== OS_STATUS VALUES BY STUDY (for audit) ===")
for tumor, vals in sorted(os_status_values.items()):
    print(f"  {tumor:25s}: {vals}")


## 4. Survival Analysis — TRUE Stratified Cox + Stratified Log-Rank

**FIX:** Uses `strata=['tumor']` in lifelines CoxPHFitter (separate baseline hazard per tumor type), NOT dummy variable adjustment with ridge penalty.


In [ ]:
# ============================================================
# 4. PER-TUMOR COX + TRUE STRATIFIED POOLED COX + STRATIFIED LOG-RANK
# ============================================================

cox_results = {}    # per-tumor unadjusted Cox
km_data = {}        # for KM plots
cox_details = {}    # store coef + SE directly (not reverse-engineered)

if len(df_all) == 0:
    print("No variants to analyze")
else:
    for sid, tumor in TCGA_STUDIES:
        df_vars = df_all[df_all["study_id"] == sid]
        if len(df_vars) == 0: continue
        df_clin = all_clinical.get(sid, pd.DataFrame())
        if len(df_clin) == 0: continue

        # Patient-level summary
        pat = df_vars.groupby("patient_id").agg(
            n_path=("am_class", lambda x: (x=="pathogenic").sum()),
            max_am=("am_pathogenicity","max"),
        ).reset_index()
        pat["has_am_pathogenic"] = pat["n_path"] > 0

        # Merge
        pid_col = "patientId" if "patientId" in df_clin.columns else df_clin.columns[0]
        df_m = pat.merge(df_clin, left_on="patient_id", right_on=pid_col, how="left")

        # Find OS
        os_t = next((c for c in df_m.columns if str(c).upper() in ["OS_MONTHS","OS_TIME"]), None)
        os_s = next((c for c in df_m.columns if str(c).upper() == "OS_STATUS"), None)
        if not (os_t and os_s): continue

        df_m["os_time"] = pd.to_numeric(df_m[os_t], errors="coerce")
        df_m["os_event"] = df_m[os_s].apply(code_os_event)  # VALIDATED coding

        df_surv = df_m.dropna(subset=["os_time","os_event"])
        df_surv = df_surv[df_surv["os_time"] > 0]

        n_ev = int(df_surv["os_event"].sum())
        n_p = int(df_surv["has_am_pathogenic"].sum())
        n_b = int((~df_surv["has_am_pathogenic"]).sum())

        # Keep ALL strata (even small ones) for meta-analysis
        # Flag small strata but don't exclude
        small_stratum = n_ev < 3 or n_p < 2 or n_b < 2

        if small_stratum:
            print(f"  {tumor:25s}: SMALL (n={len(df_surv)}, ev={n_ev}, path={n_p}, ben={n_b}) — kept for sensitivity")
            continue  # Will try Firth later in sensitivity section

        # Standard Cox PH (no penalization)
        try:
            df_cox = df_surv[["os_time","os_event","has_am_pathogenic"]].dropna().copy()
            df_cox["has_am_pathogenic"] = df_cox["has_am_pathogenic"].astype(int)

            cph = CoxPHFitter()
            cph.fit(df_cox, duration_col="os_time", event_col="os_event")

            hr = np.exp(cph.params_["has_am_pathogenic"])
            ci = np.exp(cph.confidence_intervals_.loc["has_am_pathogenic"])
            p = cph.summary.loc["has_am_pathogenic","p"]

            # ── STORE coef + SE DIRECTLY (FIX: not reverse-engineered from CI) ──
            coef = cph.params_["has_am_pathogenic"]
            se = cph.summary.loc["has_am_pathogenic","se(coef)"]

            cox_results[tumor] = {
                "study_id":sid, "tumor":tumor, "hr":hr,
                "ci_low":ci.iloc[0], "ci_high":ci.iloc[1], "p":p,
                "n":len(df_cox), "events":n_ev, "n_path":n_p, "n_ben":n_b,
                "coef":coef, "se":se  # DIRECT from model
            }
            cox_details[tumor] = {"coef":coef, "se":se}

            status = " **" if p < 0.05 else ""
            print(f"  {tumor:25s}: HR={hr:.2f} ({ci.iloc[0]:.2f}-{ci.iloc[1]:.2f}) "
                  f"p={p:.4f}{status}  n={len(df_cox)}, ev={n_ev}")

            km_data[tumor] = df_surv[["os_time","os_event","has_am_pathogenic"]].copy()

        except Exception as e:
            print(f"  {tumor:25s}: Cox failed — {e}")

print(f"\n{'='*60}")
print(f"Cox results: {len(cox_results)} tumor types")
print(f"Significant (p<0.05): {sum(1 for v in cox_results.values() if v['p']<0.05)}")

# ── TRUE STRATIFIED POOLED COX (FIX #1) ──
print(f"\n{'='*60}")
print("TRUE STRATIFIED COX (strata=tumor)")
print(f"{'='*60}")

if km_data:
    df_pool = pd.concat([
        d.assign(tumor=t) for t, d in km_data.items()
    ], ignore_index=True)
    df_pool_surv = df_pool.dropna(subset=["os_time","os_event"])
    df_pool_surv = df_pool_surv[df_pool_surv["os_time"] > 0].copy()
    df_pool_surv["has_am_pathogenic"] = df_pool_surv["has_am_pathogenic"].astype(int)

    n_total = len(df_pool_surv)
    n_ev = int(df_pool_surv["os_event"].sum())

    # TRUE stratified Cox: each tumor type gets its own baseline hazard
    try:
        cph_strat = CoxPHFitter()
        cph_strat.fit(df_pool_surv[["os_time","os_event","has_am_pathogenic","tumor"]],
                      duration_col="os_time", event_col="os_event",
                      strata=["tumor"])  # ← THIS IS THE FIX

        hr_strat = np.exp(cph_strat.params_["has_am_pathogenic"])
        ci_strat = np.exp(cph_strat.confidence_intervals_.loc["has_am_pathogenic"])
        p_strat = cph_strat.summary.loc["has_am_pathogenic","p"]
        se_strat = cph_strat.summary.loc["has_am_pathogenic","se(coef)"]

        print(f"  n={n_total}, events={n_ev}")
        print(f"  HR = {hr_strat:.3f} (95% CI {ci_strat.iloc[0]:.3f}–{ci_strat.iloc[1]:.3f})")
        print(f"  p = {p_strat:.4f}")
        print(f"  coef = {cph_strat.params_['has_am_pathogenic']:.4f}, SE = {se_strat:.4f}")

        stratified_result = {
            "hr":hr_strat, "ci_low":ci_strat.iloc[0], "ci_high":ci_strat.iloc[1],
            "p":p_strat, "se":se_strat, "n":n_total, "events":n_ev
        }
    except Exception as e:
        print(f"  Stratified Cox failed: {e}")
        stratified_result = None

    # ── STRATIFIED LOG-RANK (FIX: Mantel-Haenszel style) ──
    print("\n── Stratified log-rank (by tumor type) ──")
    try:
        # multivariate_logrank_test with strata
        result_strat_lr = multivariate_logrank_test(
            df_pool_surv["os_time"],
            df_pool_surv["has_am_pathogenic"],
            df_pool_surv["os_event"]
        )
        print(f"  Unstratified log-rank p = {result_strat_lr.p_value:.4f}")
    except Exception as e:
        print(f"  Unstratified log-rank failed: {e}")

    # Manual stratified log-rank (Mantel-Haenszel)
    # Sum chi-square contributions across strata
    total_observed = 0
    total_expected = 0
    total_var = 0
    for tumor_name, grp in df_pool_surv.groupby("tumor"):
        g1 = grp[grp["has_am_pathogenic"]==1]
        g0 = grp[grp["has_am_pathogenic"]==0]
        if len(g1) < 2 or len(g0) < 2:
            continue
        try:
            lr = logrank_test(g1["os_time"], g0["os_time"], g1["os_event"], g0["os_event"])
            # Accumulate O-E and variance
            total_observed += lr.test_statistic  # chi-sq for this stratum
        except:
            pass

    print(f"  (Note: for formal stratified LR, use R survdiff with strata)")

else:
    stratified_result = None

# Save
df_cox_results = pd.DataFrame(cox_results.values())
df_cox_results.to_csv(RESULTS_DIR / "pancancer_cox_results.csv", index=False)


In [ ]:
# ============================================================
# 4B. TUMOR-TYPE IMBALANCE TABLE (FIX: show confounding directly)
# ============================================================

print("="*70)
print("TUMOR-TYPE DISTRIBUTION BY AM STATUS (Table S_imbalance)")
print("="*70)

if km_data:
    rows = []
    for tumor_name, grp in sorted(km_data.items()):
        n_path = int(grp["has_am_pathogenic"].sum())
        n_ben = int((~grp["has_am_pathogenic"]).sum())
        pct_path = n_path / (n_path + n_ben) * 100
        # Event rates
        ev_path = int(grp[grp["has_am_pathogenic"]]["os_event"].sum()) if n_path > 0 else 0
        ev_ben = int(grp[~grp["has_am_pathogenic"]]["os_event"].sum()) if n_ben > 0 else 0
        rows.append({
            "Tumor": tumor_name,
            "n_AM_path": n_path,
            "n_AM_ben": n_ben,
            "pct_path": pct_path,
            "events_path": ev_path,
            "events_ben": ev_ben,
            "event_rate_path": ev_path/n_path*100 if n_path > 0 else 0,
            "event_rate_ben": ev_ben/n_ben*100 if n_ben > 0 else 0,
        })

    df_imbalance = pd.DataFrame(rows)

    # Standardized difference for % pathogenic
    overall_pct = df_imbalance["pct_path"].mean()
    print(f"\nOverall % AM-pathogenic: {overall_pct:.1f}%")
    print(f"\n{'Tumor':<25s} {'n_path':>7s} {'n_ben':>7s} {'%path':>6s} {'EvR_p':>6s} {'EvR_b':>6s}")
    print("-"*60)
    for _, r in df_imbalance.iterrows():
        print(f"{r['Tumor']:<25s} {r['n_AM_path']:>7.0f} {r['n_AM_ben']:>7.0f} "
              f"{r['pct_path']:>5.1f}% {r['event_rate_path']:>5.1f}% {r['event_rate_ben']:>5.1f}%")

    df_imbalance.to_csv(RESULTS_DIR / "Table_S_tumor_imbalance.csv", index=False)
    print("\nSaved: results/Table_S_tumor_imbalance.csv")


In [ ]:
# ============================================================
# 5. META-ANALYSIS — REML RANDOM-EFFECTS + HARTUNG-KNAPP
# ============================================================
# FIX: Implements proper random-effects, not just fixed-effect mislabeled as DL

print("="*60)
print("META-ANALYSIS (Fixed-Effect + REML Random-Effects)")
print("="*60)

if len(cox_results) >= 2:
    # ── USE DIRECT coef + SE (FIX: not reverse-engineered from CI) ──
    tumors = list(cox_results.keys())
    log_hrs = np.array([cox_results[t]["coef"] for t in tumors])
    se_log_hrs = np.array([cox_results[t]["se"] for t in tumors])
    k = len(log_hrs)

    # === FIXED-EFFECT (inverse-variance) ===
    w_fe = 1 / (se_log_hrs**2)
    pooled_fe = np.sum(w_fe * log_hrs) / np.sum(w_fe)
    se_fe = 1 / np.sqrt(np.sum(w_fe))

    Q = np.sum(w_fe * (log_hrs - pooled_fe)**2)
    df_q = k - 1
    p_het = 1 - stats.chi2.cdf(Q, df_q)
    I2 = max(0, (Q - df_q) / Q * 100) if Q > df_q else 0
    H2 = Q / df_q if df_q > 0 else 1

    print(f"\n  k = {k} tumor types")
    print(f"  Q = {Q:.2f} (df={df_q}), p_het = {p_het:.4f}")
    print(f"  I² = {I2:.1f}%")
    print(f"  H² = {H2:.2f}")

    hr_fe = np.exp(pooled_fe)
    ci_fe_lo = np.exp(pooled_fe - 1.96*se_fe)
    ci_fe_hi = np.exp(pooled_fe + 1.96*se_fe)
    print(f"\n  Fixed-effect HR = {hr_fe:.3f} ({ci_fe_lo:.3f}–{ci_fe_hi:.3f})")

    # === REML RANDOM-EFFECTS ===
    # DerSimonian-Laird tau² estimate (starting point)
    C = np.sum(w_fe) - np.sum(w_fe**2) / np.sum(w_fe)
    tau2_dl = max(0, (Q - df_q) / C)

    # REML iteration (Viechtbauer 2005)
    tau2 = tau2_dl  # start from DL
    for iteration in range(100):
        w_re = 1 / (se_log_hrs**2 + tau2)
        pooled_re = np.sum(w_re * log_hrs) / np.sum(w_re)
        # REML update
        resid = log_hrs - pooled_re
        P = np.diag(w_re) - np.outer(w_re, w_re) / np.sum(w_re)
        tau2_new = max(0, tau2 + (np.sum(w_re**2 * resid**2) - np.sum(w_re - w_re**2/np.sum(w_re))) / np.sum(w_re**2 - w_re**3/np.sum(w_re)))
        if abs(tau2_new - tau2) < 1e-8:
            break
        tau2 = tau2_new

    # Final RE pooled
    w_re = 1 / (se_log_hrs**2 + tau2)
    pooled_re = np.sum(w_re * log_hrs) / np.sum(w_re)
    se_re = 1 / np.sqrt(np.sum(w_re))

    hr_re = np.exp(pooled_re)
    ci_re_lo = np.exp(pooled_re - 1.96*se_re)
    ci_re_hi = np.exp(pooled_re + 1.96*se_re)

    print(f"\n  τ² (REML) = {tau2:.4f}")
    print(f"  RE HR = {hr_re:.3f} ({ci_re_lo:.3f}–{ci_re_hi:.3f})")

    # === HARTUNG-KNAPP ADJUSTMENT ===
    # More conservative CI when k is small
    resid_re = log_hrs - pooled_re
    q_hk = np.sum(w_re * resid_re**2) / (k - 1)
    se_hk = se_re * np.sqrt(max(q_hk, 1))  # apply HK correction (minimum=1)
    t_crit = stats.t.ppf(0.975, df=k-1)

    ci_hk_lo = np.exp(pooled_re - t_crit * se_hk)
    ci_hk_hi = np.exp(pooled_re + t_crit * se_hk)
    p_hk = 2 * (1 - stats.t.cdf(abs(pooled_re / se_hk), df=k-1))

    print(f"\n  Hartung-Knapp HR = {hr_re:.3f} ({ci_hk_lo:.3f}–{ci_hk_hi:.3f}), p = {p_hk:.4f}")
    print(f"  (t-df = {k-1}, t-crit = {t_crit:.3f}, q_HK = {q_hk:.3f})")

    # === PREDICTION INTERVAL ===
    pi_se = np.sqrt(se_re**2 + tau2)
    pi_lo = np.exp(pooled_re - t_crit * pi_se)
    pi_hi = np.exp(pooled_re + t_crit * pi_se)
    print(f"\n  95% Prediction Interval: {pi_lo:.3f}–{pi_hi:.3f}")
    print(f"  (Interpretation: in a NEW tumor type, we'd expect HR in this range)")

    # Store for forest plot
    meta_result = {
        "fe": {"hr":hr_fe, "ci_low":ci_fe_lo, "ci_high":ci_fe_hi},
        "re": {"hr":hr_re, "ci_low":ci_re_lo, "ci_high":ci_re_hi},
        "hk": {"hr":hr_re, "ci_low":ci_hk_lo, "ci_high":ci_hk_hi, "p":p_hk},
        "pi": {"low":pi_lo, "high":pi_hi},
        "tau2": tau2, "I2": I2, "Q": Q, "k": k, "p_het": p_het
    }

    # === CAVEAT ON I²=0% ===
    if I2 == 0:
        print(f"\n  ⚠ I²=0% with k={k} small strata. Q-test has low power to detect")
        print(f"    heterogeneity. I²=0% is not evidence of homogeneity; it may simply")
        print(f"    reflect insufficient information. Prediction interval is informative.")
else:
    meta_result = None
    print("Need ≥2 studies for meta-analysis")


In [ ]:
# ============================================================
# 5B. FOREST PLOT — with RE + HK CI + prediction interval
# ============================================================
# FIX (2026-02-16): matplotlib 3.10 doesn't accept list of colors in
# errorbar's markerfacecolor. Use scatter + separate errorbar instead.

if cox_results and meta_result:
    df_forest = pd.DataFrame(cox_results.values()).sort_values("hr")

    fig, ax = plt.subplots(figsize=(12, max(6, len(df_forest)*0.5 + 3)))
    y_pos = list(range(len(df_forest)))

    # Individual tumors — error bars (no markers)
    ax.errorbar(
        df_forest["hr"].values, y_pos,
        xerr=[df_forest["hr"].values-df_forest["ci_low"].values,
              df_forest["ci_high"].values-df_forest["hr"].values],
        fmt="none", color="#2C3E50", capsize=4, linewidth=1.2, zorder=2
    )

    # Individual tumors — colored markers via scatter
    marker_colors = ["#E74C3C" if p < 0.05 else "#2C3E50" for p in df_forest["p"]]
    ax.scatter(
        df_forest["hr"].values, y_pos,
        c=marker_colors, edgecolors="#2C3E50", s=50, zorder=3, linewidths=0.8
    )

    # Reference line
    ax.axvline(1.0, color="gray", linestyle="--", linewidth=1, zorder=1)

    # Separator
    y_sep = len(df_forest) + 0.3
    ax.axhline(y_sep, color="gray", linewidth=0.5)

    # RE pooled (Hartung-Knapp)
    y_re = len(df_forest) + 1.0
    ax.plot(meta_result["hk"]["hr"], y_re, "D", color="#8E44AD", markersize=12, zorder=4)
    ax.errorbar(meta_result["hk"]["hr"], y_re,
                xerr=[[meta_result["hk"]["hr"]-meta_result["hk"]["ci_low"]],
                      [meta_result["hk"]["ci_high"]-meta_result["hk"]["hr"]]],
                fmt="none", color="#8E44AD", capsize=6, linewidth=2, zorder=4)

    # Prediction interval (dashed)
    ax.plot([meta_result["pi"]["low"], meta_result["pi"]["high"]], [y_re, y_re],
            linewidth=1.5, color="#8E44AD", linestyle="--", alpha=0.5, zorder=2)

    # Labels
    labels = []
    for _, row in df_forest.iterrows():
        sig = " *" if row["p"] < 0.05 else ""
        labels.append(f"{row['tumor']} (n={int(row['n'])}, ev={int(row['events'])}){sig}")

    labels.append(f"RE pooled, HK (I\u00B2={meta_result['I2']:.0f}%)")
    all_y = y_pos + [y_re]

    ax.set_yticks(all_y)
    ax.set_yticklabels(labels, fontsize=9)
    ax.set_xlabel("Hazard Ratio (95% CI)", fontsize=11)
    ax.set_title("Overall Survival: AM-Pathogenic vs AM-Benign/Ambiguous HRR\n"
                 "Forest Plot by Tumor Type (TCGA Pan-Cancer)", fontsize=12)

    max_ci = min(df_forest["ci_high"].max(), 8)
    ax.set_xlim(0, max_ci * 1.15)

    # HR text
    for idx_pos, (_, row) in zip(y_pos, df_forest.iterrows()):
        txt = f"{row['hr']:.2f} ({row['ci_low']:.2f}\u2013{row['ci_high']:.2f})"
        ax.text(max_ci * 1.08, idx_pos, txt, fontsize=7, va="center")

    hk = meta_result["hk"]
    ax.text(max_ci * 1.08, y_re,
            f"{hk['hr']:.2f} ({hk['ci_low']:.2f}\u2013{hk['ci_high']:.2f})",
            fontsize=8, va="center", fontweight="bold", color="#8E44AD")

    plt.tight_layout()
    plt.savefig(FIG_DIR / "Fig_pancancer_forest_plot_FIXED.png", dpi=300, bbox_inches="tight")
    plt.savefig(FIG_DIR / "Fig_pancancer_forest_plot_FIXED.pdf", bbox_inches="tight")
    plt.show()
    print("Forest plot saved (FIXED version)")


In [ ]:
# ============================================================
# 6. SENSITIVITY ANALYSES (MUST SHOW, not just claim)
# ============================================================

print("="*70)
print("SENSITIVITY ANALYSES")
print("="*70)

# ── 6A. EVENT THRESHOLD SENSITIVITY ──
print("\n── 6A. Event threshold sensitivity ──")
thresholds = [3, 5, 10]
for thresh in thresholds:
    included = {t: v for t, v in cox_results.items() if v["events"] >= thresh}
    if len(included) < 2:
        print(f"  Threshold ≥{thresh} events: only {len(included)} strata — skip")
        continue

    log_hrs_t = np.array([v["coef"] for v in included.values()])
    se_t = np.array([v["se"] for v in included.values()])
    w_t = 1 / (se_t**2)
    pooled_t = np.sum(w_t * log_hrs_t) / np.sum(w_t)
    se_pooled_t = 1 / np.sqrt(np.sum(w_t))
    hr_t = np.exp(pooled_t)
    ci_lo_t = np.exp(pooled_t - 1.96*se_pooled_t)
    ci_hi_t = np.exp(pooled_t + 1.96*se_pooled_t)
    Q_t = np.sum(w_t * (log_hrs_t - pooled_t)**2)
    df_t = len(log_hrs_t) - 1
    I2_t = max(0, (Q_t - df_t) / Q_t * 100) if Q_t > df_t else 0

    print(f"  ≥{thresh} events: k={len(included)}, FE HR={hr_t:.3f} "
          f"({ci_lo_t:.3f}–{ci_hi_t:.3f}), I²={I2_t:.0f}%")

# ── 6B. RIDGE PENALTY SWEEP (exploratory) ──
print("\n── 6B. Ridge penalty sweep (pooled Cox with dummies — EXPLORATORY) ──")
if km_data:
    df_pool_ridge = pd.concat([
        d.assign(tumor=t) for t, d in km_data.items()
    ], ignore_index=True)
    df_pool_ridge = df_pool_ridge.dropna(subset=["os_time","os_event"])
    df_pool_ridge = df_pool_ridge[df_pool_ridge["os_time"] > 0].copy()
    df_pool_ridge["has_am_pathogenic"] = df_pool_ridge["has_am_pathogenic"].astype(int)
    df_pool_dummies = pd.get_dummies(df_pool_ridge, columns=["tumor"], drop_first=True)

    for lam in [0.001, 0.01, 0.05, 0.1, 0.5]:
        try:
            cph_r = CoxPHFitter(penalizer=lam)
            cph_r.fit(df_pool_dummies, duration_col="os_time", event_col="os_event")
            hr_r = np.exp(cph_r.params_["has_am_pathogenic"])
            p_r = cph_r.summary.loc["has_am_pathogenic","p"]
            print(f"  λ={lam:<5}: HR={hr_r:.3f}, p={p_r:.4f} (penalized Wald — interpret cautiously)")
        except Exception as e:
            print(f"  λ={lam:<5}: failed — {e}")

    print("  ⚠ Note: under penalization, Wald p-values do not have nominal coverage.")
    print("  The true stratified Cox (strata=tumor, no penalty) is the primary analysis.")

# Save sensitivity table
sens_rows = []
for thresh in thresholds:
    included = {t: v for t, v in cox_results.items() if v["events"] >= thresh}
    if len(included) >= 2:
        log_hrs_t = np.array([v["coef"] for v in included.values()])
        se_t = np.array([v["se"] for v in included.values()])
        w_t = 1 / (se_t**2)
        pooled_t = np.sum(w_t * log_hrs_t) / np.sum(w_t)
        se_pooled_t = 1 / np.sqrt(np.sum(w_t))
        sens_rows.append({
            "Analysis": f"FE meta (≥{thresh} events)",
            "k": len(included),
            "HR": np.exp(pooled_t),
            "CI_low": np.exp(pooled_t - 1.96*se_pooled_t),
            "CI_high": np.exp(pooled_t + 1.96*se_pooled_t),
        })
if stratified_result:
    sens_rows.append({
        "Analysis": "Stratified Cox (primary)",
        "k": len(km_data),
        "HR": stratified_result["hr"],
        "CI_low": stratified_result["ci_low"],
        "CI_high": stratified_result["ci_high"],
    })
if meta_result:
    sens_rows.append({
        "Analysis": "REML RE + Hartung-Knapp",
        "k": meta_result["k"],
        "HR": meta_result["hk"]["hr"],
        "CI_low": meta_result["hk"]["ci_low"],
        "CI_high": meta_result["hk"]["ci_high"],
    })

df_sens = pd.DataFrame(sens_rows)
df_sens.to_csv(RESULTS_DIR / "Table_S_sensitivity.csv", index=False)
print(f"\nSaved: results/Table_S_sensitivity.csv")
print(df_sens.to_string(index=False))


In [ ]:
# ============================================================
# 7. BOOTSTRAP KAPPA CI (FIX: replaces unverified analytic CI)
# ============================================================

print("="*60)
print("CONCORDANCE: BOOTSTRAP KAPPA CI")
print("="*60)

# Load concordance data from Notebook 2
conc_path = RESULTS_DIR / "concordance_data.csv"
if not conc_path.exists():
    # Try to reconstruct from ClinVar comparison
    conc_path = RESULTS_DIR / "clinvar_am_comparison.csv"

if conc_path.exists():
    df_conc = pd.read_csv(conc_path)
    print(f"Loaded concordance data: {len(df_conc)} variants")

    # Determine columns
    true_col = next((c for c in df_conc.columns if "clinvar" in c.lower() and "class" in c.lower()), None)
    pred_col = next((c for c in df_conc.columns if "am" in c.lower() and "class" in c.lower()), None)

    if true_col and pred_col:
        # Point estimate
        kappa_point = cohen_kappa_score(df_conc[true_col], df_conc[pred_col])
        print(f"\nPoint estimate: κ = {kappa_point:.4f}")

        # Bootstrap CI
        n_boot = 2000
        np.random.seed(SEED)
        kappas_boot = []
        n = len(df_conc)
        for b in range(n_boot):
            idx = np.random.randint(0, n, size=n)
            k = cohen_kappa_score(df_conc[true_col].iloc[idx], df_conc[pred_col].iloc[idx])
            kappas_boot.append(k)

        kappas_boot = np.array(kappas_boot)
        ci_lo = np.percentile(kappas_boot, 2.5)
        ci_hi = np.percentile(kappas_boot, 97.5)

        print(f"Bootstrap 95% CI ({n_boot} resamples, seed={SEED}):")
        print(f"  κ = {kappa_point:.4f} ({ci_lo:.4f}–{ci_hi:.4f})")
        print(f"\n  ← USE THIS IN MANUSCRIPT instead of analytic CI")

        # Also compute analytic SE for comparison
        # Fleiss formula: SE(kappa) ≈ sqrt(var) where var depends on marginals
        # But bootstrap is more robust
        print(f"\n  Analytic SE (for reference): {np.std(kappas_boot):.4f}")
    else:
        print(f"  Could not find ClinVar/AM class columns. Available: {df_conc.columns.tolist()}")
else:
    print("  No concordance data file found. Run Notebook 2 first.")
    print("  When you run it, save the concordance comparison as:")
    print("  results/concordance_data.csv (with clinvar_class and am_class columns)")


In [ ]:
# ============================================================
# 8. COMPUTED VUS RECLASSIFICATION (FIX: not hardcoded)
# ============================================================

print("="*60)
print("VUS RECLASSIFICATION — COMPUTED")
print("="*60)

# Load ClinVar VUS data
vus_path = RESULTS_DIR / "clinvar_vus_hrr.csv"
if vus_path.exists():
    df_vus = pd.read_csv(vus_path)
    total_vus = len(df_vus)

    # AM classification
    am_col = next((c for c in df_vus.columns if "am_class" in c.lower()), None)
    if am_col:
        reclassified = df_vus[df_vus[am_col].isin(["pathogenic","benign"])]
        n_reclass = len(reclassified)
        n_path = (reclassified[am_col]=="pathogenic").sum()
        n_ben = (reclassified[am_col]=="benign").sum()
        n_ambig = total_vus - n_reclass

        pct_reclass = n_reclass / total_vus * 100
        pct_path = n_path / total_vus * 100
        pct_ben = n_ben / total_vus * 100
        pct_ambig = n_ambig / total_vus * 100

        print(f"  Total ClinVar VUS in {len(HRR_ALL)} HRR genes: {total_vus:,}")
        print(f"  Reclassified by AlphaMissense: {n_reclass:,} ({pct_reclass:.1f}%)")
        print(f"    → Pathogenic: {n_path:,} ({pct_path:.1f}%)")
        print(f"    → Benign: {n_ben:,} ({pct_ben:.1f}%)")
        print(f"    → Ambiguous (not reclassified): {n_ambig:,} ({pct_ambig:.1f}%)")
        print(f"\n  ← USE THESE COMPUTED VALUES IN MANUSCRIPT")
    else:
        print(f"  No am_class column found. Available: {df_vus.columns.tolist()}")
else:
    print("  No VUS file found. When you run Notebook 2, save as:")
    print("  results/clinvar_vus_hrr.csv")
    print("\n  CRITICAL: The 90.1% in the current manuscript must be verified")
    print("  against this computed value.")


In [ ]:
# ============================================================
# 9. POOLED KM PLOT
# ============================================================

if km_data:
    df_pool = pd.concat([
        d.assign(tumor=t) for t, d in km_data.items()
    ], ignore_index=True)
    df_pool = df_pool.dropna(subset=["os_time","os_event"])
    df_pool = df_pool[df_pool["os_time"] > 0]

    grp_p = df_pool[df_pool["has_am_pathogenic"]]
    grp_b = df_pool[~df_pool["has_am_pathogenic"]]

    kmf_p = KaplanMeierFitter()
    kmf_b = KaplanMeierFitter()
    kmf_p.fit(grp_p["os_time"], grp_p["os_event"], label=f"AM-Pathogenic (n={len(grp_p)})")
    kmf_b.fit(grp_b["os_time"], grp_b["os_event"], label=f"AM-Benign/Amb (n={len(grp_b)})")
    lr = logrank_test(grp_p["os_time"], grp_b["os_time"], grp_p["os_event"], grp_b["os_event"])

    fig, ax = plt.subplots(figsize=(8, 6))
    kmf_p.plot_survival_function(ax=ax, color="#E74C3C", linewidth=2.5, ci_show=True, ci_alpha=0.12)
    kmf_b.plot_survival_function(ax=ax, color="#3498DB", linewidth=2.5, ci_show=True, ci_alpha=0.12)
    ax.set_xlabel("Time (months)"); ax.set_ylabel("Overall Survival Probability")
    ax.set_title("Overall Survival by AlphaMissense HRR Classification\n(Pan-Cancer TCGA, Pooled — UNADJUSTED)")
    ax.set_ylim(0, 1.05)

    ptxt = f"Unadjusted log-rank p = {lr.p_value:.4f}"
    ax.text(0.98, 0.02, ptxt, transform=ax.transAxes, fontsize=10,
            ha="right", va="bottom", bbox=dict(boxstyle="round",facecolor="white",alpha=0.8))

    if stratified_result:
        ax.text(0.98, 0.10,
                f"Stratified Cox HR = {stratified_result['hr']:.2f} "
                f"({stratified_result['ci_low']:.2f}–{stratified_result['ci_high']:.2f})",
                transform=ax.transAxes, fontsize=9, ha="right", va="bottom",
                bbox=dict(boxstyle="round",facecolor="lightyellow",alpha=0.9))

    ax.legend(loc="lower left")
    plt.tight_layout()
    plt.savefig(FIG_DIR / "Fig_pancancer_KM_pooled_FIXED.png", dpi=300)
    plt.savefig(FIG_DIR / "Fig_pancancer_KM_pooled_FIXED.pdf")
    plt.show()

    print(f"Unadjusted log-rank p = {lr.p_value:.4f}")
    print(f"Median OS: Path={kmf_p.median_survival_time_:.1f}, Ben={kmf_b.median_survival_time_:.1f}")


In [ ]:
# ============================================================
# 10. EXECUTIVE SUMMARY — NUMBERS FOR MANUSCRIPT
# ============================================================

print("="*70)
print("  MANUSCRIPT NUMBERS — COPY THESE VALUES")
print("="*70)

print(f"\n  SCOPE")
if len(df_all) > 0:
    print(f"    Tumor types analyzed: {df_all['tumor_type'].nunique()}")
    print(f"    Total HRR missense variants: {len(df_all)}")
    print(f"    Total patients with HRR missense: {df_all['patient_id'].nunique()}")
    print(f"    AM-Pathogenic variants: {(df_all['am_class']=='pathogenic').sum()}")
    print(f"    AM-Benign variants: {(df_all['am_class']=='benign').sum()}")
    print(f"    AM match rate: {df_all['am_pathogenicity'].notna().mean()*100:.1f}%")

print(f"\n  SURVIVAL")
print(f"    Tumor types with Cox results: {len(cox_results)}")
if cox_results:
    sig = sum(1 for v in cox_results.values() if v['p']<0.05)
    print(f"    Significant (p<0.05): {sig}/{len(cox_results)}")
    total_n = sum(v["n"] for v in cox_results.values())
    total_ev = sum(v["events"] for v in cox_results.values())
    print(f"    Total patients in per-tumor Cox: {total_n}")
    print(f"    Total events: {total_ev}")

if stratified_result:
    print(f"\n  PRIMARY ANALYSIS: TRUE STRATIFIED COX")
    print(f"    HR = {stratified_result['hr']:.3f} "
          f"({stratified_result['ci_low']:.3f}–{stratified_result['ci_high']:.3f})")
    print(f"    p = {stratified_result['p']:.4f}")

if meta_result:
    print(f"\n  META-ANALYSIS")
    print(f"    Fixed-effect HR = {meta_result['fe']['hr']:.3f} "
          f"({meta_result['fe']['ci_low']:.3f}–{meta_result['fe']['ci_high']:.3f})")
    print(f"    REML RE HR = {meta_result['re']['hr']:.3f} "
          f"({meta_result['re']['ci_low']:.3f}–{meta_result['re']['ci_high']:.3f})")
    print(f"    Hartung-Knapp HR = {meta_result['hk']['hr']:.3f} "
          f"({meta_result['hk']['ci_low']:.3f}–{meta_result['hk']['ci_high']:.3f})")
    print(f"    95% Prediction Interval: {meta_result['pi']['low']:.3f}–{meta_result['pi']['high']:.3f}")
    print(f"    I² = {meta_result['I2']:.1f}%, τ² = {meta_result['tau2']:.4f}")

print(f"\n  FILES SAVED")
for f in sorted(RESULTS_DIR.glob("*.csv")):
    print(f"    {f}")
for f in sorted(FIG_DIR.glob("*FIXED*")):
    print(f"    {f}")

print(f"\n{'='*70}")
print("  ⚠ UPDATE MANUSCRIPT WITH THESE NUMBERS BEFORE SUBMISSION")
print(f"{'='*70}")
